Bộ dữ liệu chứa thông tin hai tháng về tất cả các yêu cầu HTTP gửi đến máy
chủ WWW của 1 công ty. Nhật ký đầu tiên được thu thập từ 00:00:00 ngày 1
tháng 7 năm 1995 đến 23:59:59 ngày 31 tháng 7 năm 1995, tổng cộng là 31
ngày. Nhật ký thứ hai được thu thập từ 00:00:00 ngày 1 tháng 8 năm 1995 đến
23:59:59 ngày 31 tháng 8 năm 1995, tổng cộng là 31 ngày. Các dấu thời gian có
độ phân giải 1 giây. Lưu ý rằng từ 14:52:01 ngày 01/08/1995 đến 04:36:13 ngày
03/08/1995 không có truy cập nào được ghi lại, do máy chủ Web đã bị tắt vì
Bão.
- Thành phần dữ liệu:Dữ liệu gốc ở định dạng ASCII, với mỗi dòng tương ứng
một yêu cầu. Thí sinh cần xử lý và trích xuất các trường thông tin:


- Host (Nguồn): Địa chỉ IP hoặc tên miền của máy khách gửi yêu cầu (Ví
dụ: 199.72.81.55).
- Timestamp (Thời gian): Thời điểm yêu cầu được ghi nhận (Ví dụ:
[01/Jul/1995:00:00:01 -0400]). Đây là trường quan trọng nhất để tạo
chuỗi thời gian.
- Request (Yêu cầu): Chứa phương thức (GET/POST), đường dẫn tài
nguyên (URL) và giao thức (Ví dụ: "GET /history/apollo/ HTTP/1.0").
- HTTP Reply Code (Trạng thái): Mã phản hồi từ máy chủ.
- Bytes in the reply (Kích thước): Dung lượng dữ liệu trả về.
- Quy định chia tập dữ liệu (Train/Test Split)
- Tập Huấn luyện (Train Set): Dữ liệu của tháng 7 và 22 ngày đầu tiên
của tháng 8.
- Tập Kiểm thử (Test Set): Dữ liệu của các ngày còn lại trong tháng 8.
- Yêu cầu tiền xử lý (Ingest + EDA):
- Pipeline đọc log, chuẩn hóa timestamp, parse fields (IP, URL, status).
- Khai thác time series: hits/sec, error rate, spike detection.

| Status                             | Meaning                         | What it means in your data                                                                         |
| ---------------------------------- | ------------------------------- | -------------------------------------------------------------------------------------------------- |
| **200 OK**                         | Request succeeded               | The page or API endpoint was returned correctly.                                                   |
| **304 Not Modified**               | Cached version is still valid   | The client already had the resource and the server said “no change”. Common for static files.      |
| **302 Found (Temporary Redirect)** | Redirect to another URL         | The server redirected the user (e.g., login, HTTPS redirect, or URL change).                       |
| **404 Not Found**                  | Resource does not exist         | Broken links, missing pages, bad URLs, or crawlers probing random paths.                           |
| **403 Forbidden**                  | Access denied                   | The client is blocked or lacks permission (IP blocked, auth required).                             |
| **500 Internal Server Error**      | Server crashed while processing | Bug, memory error, timeout, or bad backend logic.                                                  |
| **501 Not Implemented**            | Method not supported            | The server doesn’t recognize the request method (e.g., PUT on a server that only allows GET/POST). |
| **400 Bad Request**                | Malformed request               | Invalid syntax, missing parameters, or corrupted requests.                               

After Normalized: 

200 -> Success

304 -> No Change

302 -> Redirected

404 -> Not Found

Others -> Error

In [ ]:
%load_ext autoreload
%autoreload 2


In [1]:
import os

cwd = os.getcwd()

if not cwd.endswith("eda"):
    raise ValueError("Wrong path, should be in eda folder")

BASE_DIR = os.path.join(cwd, "..")

In [2]:
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

processed_dir = os.path.join(BASE_DIR, "data", "processed")

# Load data
df = pd.read_csv(os.path.join(processed_dir, "train.csv"))

# test_df = pd.read_csv(os.path.join(processed_dir, "test.csv"))


In [3]:
# Preprocessing: Convert time to datetime index
df['time'] = pd.to_datetime(df['time'])
df.set_index('time', inplace=True)
df.sort_index(inplace=True)
df['status'] = df['status'].astype(str)

print(f"Train Shape: {df.shape}")
# print(f"Test Shape: {test_df.shape}")

Train Shape: (2934932, 10)


In [ ]:
df.head(5)

In [4]:
df.columns

Index(['ip', 'identd', 'user', 'request', 'status', 'size', 'resource',
       'protocol', 'utc', 'status_label'],
      dtype='object')

In [ ]:
FIG_SIZE = (20, 5)

In [ ]:
# df.resample('5T').size().plot(figsize = FIG_SIZE)

## EDA

### Macro level Overview


In [ ]:
# df.resample('15T').size().plot(figsize = FIG_SIZE)
# df.resample('1H').size().plot(figsize = FIG_SIZE)

In [ ]:
from plot_utils import print_macro_overview

print_macro_overview(df)


### Temporal Patterns

#### Total request counts by time interval in a week

In [ ]:
from plot_utils import plot_weekly_heatmap

plot_weekly_heatmap(df, '1H')
# plot_weekly_heatmap(df, '15T')

del plot_weekly_heatmap

#### Mean success ratio, total file size, total hits per day of week and time of day

In [ ]:
from plot_utils import plot_weekly_patterns

# plot_weekly_patterns(df, interval='1H', figsize=(20, 24))
# plot_weekly_patterns(df, interval='30T', figsize=(20, 24))
plot_weekly_patterns(df, interval='15T', figsize=(20, 24))


#### Mean success ratio, total file size, total hits time of day (line plots)

In [ ]:
from plot_utils import plot_daily_profile

# See the average daily pattern (e.g. morning peak, lunch dip, etc.)
plot_daily_profile(df, interval='10T')

In [ ]:
from plot_utils import plot_daily_profile_split

fig_hits, fig_size, fig_rate= plot_daily_profile_split(df, interval='10T')


In [10]:
fig_hits
fig_hits.savefig("daily_profile_hits.png", dpi=300, bbox_inches="tight")

In [11]:
fig_size
fig_size.savefig("daily_profile_size.png", dpi=300, bbox_inches="tight")


In [12]:
fig_rate
fig_rate.savefig("daily_profile_rate.png", dpi=300, bbox_inches="tight")


### Traffic Characteristics

#### Analyse status distribution

In [ ]:
from plot_utils import analyze_status_distribution

analyze_status_distribution(df)

#### Top 10 domains that send the most request

In [ ]:
# top_10 = df.groupby('ip').select_dtypes(include=np.number).aggregate(sum).sort_values().head(10)
top_10 = df.groupby('ip').sum(numeric_only=True).sort_values('size', ascending=False).head(10)

In [ ]:
top_10

#### Line plot over time of 10 domains that sends the most number of request

### Contextual Analysis (what is requested)

#### what types of resources that are popular

In [ ]:
from plot_utils import plot_file_type_stats

plot_file_type_stats(df)

#### Top most active IPs (most request and most data downloaded)

In [ ]:
from plot_utils import plot_top_users

plot_top_users(df)

#### Status distribution breakdown

In [ ]:
from plot_utils import plot_status_breakdown

plot_status_breakdown(df)


### Bonus

#### Rolling mean

In [ ]:
from plot_utils import plot_rolling_statistics

plot_rolling_statistics(df, window='1H')


#### Anamoly

In [ ]:
from plot_utils import plot_anomaly_spikes

plot_anomaly_spikes(df, interval='1T', threshold_z=3)


In [ ]:
plot_anomaly_spikes(df, interval='15T', threshold_z=3)

In [ ]:
plot_anomaly_spikes(df, interval='1H', threshold_z=3)

### Contents and Reliability

#### Stacked Area Chart (volume of each status code over time, stacked on top of each other)

In [ ]:
from plot_utils import plot_status_evolution

plot_status_evolution(df)


#### File size histogram log scale

In [ ]:
from plot_utils import plot_size_distribution

plot_size_distribution(df)


In [ ]:
####